In [2]:
import random
import json
from useful_stuff.bots import RandBot, RdeepBot, MyBotA
from useful_stuff.game import SchnapsenGamePlayEngine

class Duel(): 
    def __init__(self, name1, name2):
        self.match_results = None
        self.total_results = {str(name1): 0, str(name2): 0}
        self.bots = [name1, name2]

    def addMatchResultToTotalResults(self):
         self.total_results = { key: self.total_results[key] + self.match_results[key] for key in self.match_results.keys()  }

    def playMatch(self, matchID):
        self.match_results = {str(self.bots[0]): 0, str(self.bots[1]): 0}
        firstBot = str(self.bots[matchID % 2])

        while all(p < 7 for p in self.match_results.values()):
            winner_id, game_points, score = self.play_game(firstBot)
            self.match_results[str(winner_id)] += game_points
            firstBot = str(winner_id)

        self.addMatchResultToTotalResults()

        return {"match-id": matchID,"match-results": self.match_results, "total-results": self.total_results}

    def play_game(self, firstBot = None):
        if str(self.bots[0]) == firstBot :
            return SchnapsenGamePlayEngine().play_game(self.bots[0], self.bots[1], random.Random(random.randint(0, 200)))
        else:
            return SchnapsenGamePlayEngine().play_game(self.bots[1], self.bots[0], random.Random(random.randint(0, 200)))

bots = [
    RandBot(rand=random.Random(random.randint(0, 200)), name="RandBot"),
    RdeepBot(num_samples = 50, depth=12, rand=random.Random(random.randint(0, 200)), name="RdeepBot"),
]

myBots = [
    RandBot(rand=random.Random(random.randint(0, 200)), name="RandomBot")
]
          
for i in range(len(myBots)):
    for j in range(len(bots)):
    
        bot1 = bots[j]
        bot2 = myBots[i]
        duel = Duel(bot1, bot2)
        results = [duel.playMatch(g + 1) for g in range(4)]

        with open(f"results_{str(bot1)}_vs_{str(bot2)}.json", "w") as file:
            json.dump(results, file)     
             
